In [1]:
import pandas as pd
import boto3
import json

# Load DWH Params from a file

In [2]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('../../covid_pipeline.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

DWH_CLUSTER_TYPE       = config.get("CLUSTER","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("CLUSTER","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("CLUSTER","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("CLUSTER","DWH_CLUSTER_IDENTIFIER")
DWH_DB                 = config.get("CLUSTER","DB_NAME")
DWH_DB_USER            = config.get("CLUSTER","DB_USER")
DWH_DB_PASSWORD        = config.get("CLUSTER","DB_PASSWORD")
DWH_PORT               = config.get("CLUSTER","DB_PORT")

DWH_IAM_ROLE_NAME      = config.get("CLUSTER", "DWH_IAM_ROLE_NAME")


(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)

pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
             })

,Param,Value
0,DWH_CLUSTER_TYPE,multi-node
1,DWH_NUM_NODES,4
2,DWH_NODE_TYPE,dc2.large
3,DWH_CLUSTER_IDENTIFIER,covid-cluster
4,DWH_DB,covid-db
5,DWH_DB_USER,covid-user
6,DWH_DB_PASSWORD,Nomorecovid1
7,DWH_PORT,5439
8,DWH_IAM_ROLE_NAME,sparkifyDataLakeRole


## Create clients for EC2, IAM, and Redshift

In [3]:
import boto3

ec2 = boto3.resource('ec2',
                       region_name='us-east-1',
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET)

iam = boto3.client('iam',
                       region_name='us-east-1',
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET)

redshift = boto3.client('redshift',
                       region_name='us-east-1',
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET)

## Create IAM Role

In [4]:
try:
    dwhRole = iam.create_role(Path='/', RoleName=DWH_IAM_ROLE_NAME,
                             Description='Allows Redshift clusters to call AWS services',
                             AssumeRolePolicyDocument=json.dumps({'Statement':
                                                                 [{'Action':'sts:AssumeRole',
                                                                 'Effect':'Allow',
                                                                 'Principal':{'Service':'redshift.amazonaws.com'}}],
                             'Version': '2012-10-17'}))
    

except Exception as e:
    print(e)


An error occurred (EntityAlreadyExists) when calling the CreateRole operation: Role with name sparkifyDataLakeRole already exists.


## Attach role policy

In [5]:
iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']

200

## Check role

In [6]:
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)

arn:aws:iam::244864396415:role/sparkifyDataLakeRole


##  Create Cluster
- Create a RedShift Cluster
- For complete arguments to `create_cluster`, see [docs](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/redshift.html#Redshift.Client.create_cluster)

In [7]:
try:
    response = redshift.create_cluster(         
        #hardware
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),
        #identifiers & credentials
        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
        #role (to allow s3 access)
        IamRoles=[roleArn]  
    )
except Exception as e:
    print(e)

## *Describe* the cluster to see its status


In [8]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

C:\Users\mitch\Anaconda3\envs\data_eng_tf\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


,Key,Value
0,ClusterIdentifier,covid-cluster
1,NodeType,dc2.large
2,ClusterStatus,creating
3,MasterUsername,covid-user
4,DBName,covid-db
5,VpcId,vpc-666bc91b
6,NumberOfNodes,4


In [9]:
import time
while True:
    myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
    if myClusterProps['ClusterStatus'] == 'available':
        print('cluster ready')
        break
    else:
        time.sleep(5)

cluster ready


## Check Endpoint

In [12]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)

DWH_ENDPOINT ::  covid-cluster.c2yxu1dbnzfx.us-east-1.redshift.amazonaws.com
DWH_ROLE_ARN ::  arn:aws:iam::244864396415:role/sparkifyDataLakeRole


## Open an incoming  TCP port to access the cluster ednpoint

In [13]:
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)

ec2.SecurityGroup(id='sg-01a0d1082413be67d')
An error occurred (InvalidPermission.Duplicate) when calling the AuthorizeSecurityGroupIngress operation: the specified rule "peer: 0.0.0.0/0, TCP, from port: 5439, to port: 5439, ALLOW" already exists


## STEP 4: Make sure you can connect to the clusterConnect to the cluster

%load_ext sql

In [ ]:
conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT,DWH_DB)
#print(conn_string)
%sql $conn_string

In [64]:
import psycopg2

config.read('../../covid_pipeline.cfg')
conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(DWH_ENDPOINT, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT))

In [88]:
("host={} dbname={} user={} password={} port={}".format(DWH_ENDPOINT, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT))

'host=covid-cluster.c2yxu1dbnzfx.us-east-1.redshift.amazonaws.com dbname=covid-db user=covid-user password=Nomorecovid1 port=5439'

In [65]:
res = conn.cursor()

In [75]:
res.execute("""CREATE  TABLE IF NOT EXISTS staging_covid_vaccine_tweets(
                                user_name TEXT,
                                user_location TEXT,
                                user_description NUMERIC,
                                user_created NUMERIC,
                                user_friends TEXT,
                                user_favourites TEXT,
                                user_verified TEXT,
                                date TEXT,
                                text TEXT,
                                hashtags TEXT,
                                source TEXT,
                                is_retweet BOOLEAN)""")
conn.commit()

In [76]:
res.execute("""create view loadview as
(select distinct tbl, trim(name) as table_name, query, starttime,
trim(filename) as input, line_number, colname, err_code,
trim(err_reason) as reason
from stl_load_errors sl, stv_tbl_perm sp
where sl.tbl = sp.id);""")
conn.commit()

DuplicateTable: relation "loadview" already exists


In [77]:
conn.commit()

In [78]:
res.execute("""copy staging_covid_vaccine_tweets from 's3://udacity-capstone-data/vaccine_tweets/covidvaccine_tweets.csv'
                          credentials 'aws_iam_role=arn:aws:iam::244864396415:role/sparkifyDataLakeRole'
                          CSV;""")
conn.commit()

InternalError_: Load into table 'staging_covid_vaccine_tweets' failed.  Check 'stl_load_errors' system table for details.


In [79]:
conn.commit()

In [80]:
res.execute("""select * from loadview where table_name='staging_covid_vaccine_tweets';""")

In [ ]:
res.fetchall()

In [84]:
res.execute("""select table_name, query, line_number, colname, starttime, 
trim(reason) as error
from loadview
where table_name ='staging_covid_vaccine_tweets'
order by line_number limit 1;""")

In [85]:
res.fetchall()

[]

In [86]:
res.execute("""SELECT errors.tbl, info.table_id::integer, info.table_id, *
FROM stl_load_errors errors
INNER JOIN svv_table_info info
  ON errors.tbl = info.table_id""")

In [87]:
res.fetchall()

[]

In [16]:
myClusterProps

{'ClusterIdentifier': 'covid-cluster',
 'NodeType': 'dc2.large',
 'ClusterStatus': 'available',
 'ClusterAvailabilityStatus': 'Available',
 'MasterUsername': 'covid-user',
 'DBName': 'covid-db',
 'Endpoint': {'Address': 'covid-cluster.c2yxu1dbnzfx.us-east-1.redshift.amazonaws.com',
  'Port': 5439},
 'ClusterCreateTime': datetime.datetime(2021, 3, 28, 23, 36, 49, 637000, tzinfo=tzutc()),
 'AutomatedSnapshotRetentionPeriod': 1,
 'ManualSnapshotRetentionPeriod': -1,
 'ClusterSecurityGroups': [],
 'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-6f969d64',
   'Status': 'active'}],
 'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
   'ParameterApplyStatus': 'in-sync'}],
 'ClusterSubnetGroupName': 'default',
 'VpcId': 'vpc-666bc91b',
 'AvailabilityZone': 'us-east-1c',
 'PreferredMaintenanceWindow': 'wed:05:30-wed:06:00',
 'PendingModifiedValues': {},
 'ClusterVersion': '1.0',
 'AllowVersionUpgrade': True,
 'NumberOfNodes': 4,
 'PubliclyAccessible': True,
 'Encrypted'

In [17]:
prettyRedshiftProps(myClusterProps)

C:\Users\mitch\Anaconda3\envs\data_eng_tf\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


,Key,Value
0,ClusterIdentifier,covid-cluster
1,NodeType,dc2.large
2,ClusterStatus,available
3,MasterUsername,covid-user
4,DBName,covid-db
5,Endpoint,"{'Address': 'covid-cluster.c2yxu1dbnzfx.us-east-1.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-666bc91b
7,NumberOfNodes,4
